# Test Processing of a single cycling data

The cycling data will include both RPTs and information during cycling. First, split up these two pieces. Then, for each piece, parse the data.

Compile information on UMBLFEB2022 formation cells

1/17/2023

In [20]:
import os, sys
from matplotlib import pyplot as plt

if os.path.basename(os.getcwd()) == 'notebooks-2023':
    os.chdir('../')
    sys.path.insert(0, 'src/')
    
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib ipympl

import src.plotter as plotter
import src.vas as vas
import src.parsers as parsers

%load_ext autoreload
%autoreload 2

plotter.initialize(plt)

In [3]:
# Initialize the helper class
vh = vas.VasHelper()

Initializing Voltaiq Analytic Studio Helper...
Initializing test records...
Initializing devices...
Done.


In [16]:
device_name = 'UMBL2022FEB_CELL152030'
parser = parsers.CyclingDataParser(device_name, vh)

# First inspect the entire dataset

In [27]:
plot_vars = ['voltage_v', 'cycle_index', 'step_index']
xvar = 'test_time_s'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(8, nrows * 3), sharex=True)

for yvar, ax in zip(plot_vars, axs):
    ax.plot(parser.df[xvar], parser.df[yvar])
    ax.set_ylabel(yvar)
    if ax != axs[-1]:
        ax.tick_params(labelbottom=False)
    
axs[-1].set_xlabel(xvar);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
df_by_cyc = parser.get_cycling_info()
df_by_cyc

,charge_capacity_ah,charge_energy_wh,discharge_capacity_ah,discharge_energy_wh,tot_charge_capacity_ah,tot_charge_energy_wh,tot_discharge_capacity_ah,tot_discharge_energy_wh,charge_cc_time_s,charge_cv_time_s,charge_rest_delta_voltage_v,discharge_cc_time_s,discharge_rest_delta_voltage_v
cycle_index,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,0.000000,0.000000,0.161482,0.511116,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,0.081936,0.268059,0.168439,0.531223,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.163858,0.548809,0.175382,0.553034,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,0.245795,0.835924,0.182333,0.575806,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,0.327736,1.124766,0.189288,0.598931,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1611,NaN,NaN,NaN,NaN,2286.403605,9194.775027,2284.192846,7860.127661,NaN,NaN,NaN,NaN,NaN
1612,NaN,NaN,NaN,NaN,2287.038497,9197.439459,2284.199794,7860.150892,NaN,NaN,NaN,NaN,NaN
1613,NaN,NaN,NaN,NaN,2288.937512,9204.618198,2286.127168,7867.248592,NaN,NaN,NaN,NaN,-0.000162


In [32]:
plot_vars = ['discharge_capacity_ah', 
             'charge_cc_time_s', 
             'charge_cv_time_s', 
             'charge_rest_delta_voltage_v',
             'discharge_cc_time_s', 
             'discharge_rest_delta_voltage_v']

xvar = 'tot_discharge_capacity_ah'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(12, nrows * 4), sharex=True)

for yvar, ax in zip(plot_vars, axs):
    ax.plot(df_by_cyc[xvar], df_by_cyc[yvar])
    if ax != axs[-1]:
        ax.tick_params(labelbottom=False)
    if ax == axs[0]:
        ax.set_title(f'{device_name} \n {yvar}')
    else:
        ax.set_title(yvar)
    
axs[-1].set_xlabel(xvar);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Process the RPT Data

In [40]:
df_by_rpt = parser.get_rpt_info()
df_by_rpt

In [42]:
fh2, axs2 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)
fh3, axs3 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)

for cycle_index, row in df_by_rpt.iterrows():
    
    data = row['dvdq_data'];
    
    axs2[0].plot(data['chg_q'], data['chg_v'])
    axs2[1].plot(data['dch_q'], data['dch_v'])
    
    axs3[0].plot(data['chg_q'], data['chg_dvdq'])
    axs3[1].plot(data['dch_q'], data['dch_dvdq'])

    axs3[0].set_ylim([0, 0.75])
    axs3[1].set_ylim([-0.75, 0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plotting Cycling and RPT data together

In [43]:
plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['c20_discharge_capacity_ah'], marker='o', label='C/20 DCH')
plt.plot(df_by_cyc['tot_discharge_capacity_ah']/2.5, df_by_cyc['discharge_capacity_ah'], label='1C DCH')
plt.legend()
plt.ylabel('Capacity (Ah)')
plt.xlabel('Throughput (Ah)')
plt.grid(False)
plt.title(device_name);

plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['deltav_3p0v_2p5hr'], marker='o', label='$\Delta$V, 3.0V, 2.5hr')
plt.legend()
plt.ylabel('$\Delta$ V')
plt.xlabel('Throughput (Ah)')
plt.grid(False)
plt.title(device_name);

plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['deltav_4p2v_12hr'], marker='o', label='$\Delta$V, 4.2V, 12hr')
plt.legend()
plt.ylabel('$\Delta$ V')
plt.xlabel('Throughput (Ah)')
plt.grid(False)
plt.title(device_name);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
plot_vars = ['voltage_v', 'cycle_index', 'step_index']
xvar = 'test_time_s'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(12, nrows * 3), sharex=True)


for yvar, ax in zip(plot_vars, axs):
    ax.plot(parser.df_rpt[xvar], parser.df_rpt[yvar], ls='', marker='o', ms=0.3, c='r')
    ax.plot(parser.df_cyc[xvar], parser.df_cyc[yvar], ls='', marker='o', ms=0.3, c='b')
    ax.set_ylabel(yvar)
    if ax != axs[-1]:
        ax.tick_params(labelbottom=False)
    
axs[-1].set_xlabel(xvar);
axs[0].set_ylim((2.7, 4.22))
axs[0].set_title(device_name);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Process HPPC data from each RPT

In [52]:
df_hppc = parser.get_hppc_info()
df_hppc

,cycle_index,throughput_ah,resistance_discharge_ohms,resistance_charge_ohms,capacity_ah
0,1,0.161482,0.086214,0.071357,0.000000
1,1,0.161482,0.065927,0.059111,0.074974
2,1,0.161482,0.057287,0.053531,0.149953
3,1,0.161482,0.050547,0.048927,0.224938
4,1,0.161482,0.048276,0.047172,0.299907
...,...,...,...,...,...
365,1593,2284.073335,0.212496,0.187780,1.049688
366,1593,2284.073335,0.212175,0.186858,1.124669
367,1593,2284.073335,0.211593,0.186080,1.199650
368,1593,2284.073335,0.211013,NaN,1.274631


In [51]:
plt.figure()

colors = list(reversed(plt.cm.jet(np.linspace(0,1,len(df_hppc['cycle_index'].unique())+1))))

for idx, cycle_index in enumerate(reversed(df_hppc['cycle_index'].unique())):
    
    curr_df = df_hppc[df_hppc['cycle_index'] == cycle_index]
    curr_throughput_ah = curr_df['throughput_ah'].head(1).values[0]
    plt.plot(curr_df['capacity_ah'], curr_df['resistance_discharge_ohms'], label=f'{curr_throughput_ah:.0f} Ah', marker='o', color=colors[idx])
    plt.plot(curr_df['capacity_ah'], curr_df['resistance_charge_ohms'], label=f'', marker='x', linestyle='', color=colors[idx])
       
plt.legend()
plt.xlabel('Capacity (Ah)')
plt.ylabel('10s Resistance ($\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '10s Resistance ($\\Omega$)')